In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer


from xgboost import XGBClassifier
from xgboost import plot_importance

import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
df = pd.read_csv('../data/raw_postpro.csv', encoding='cp949')
# DataFrame인 df가 있다고 가정합니다.
df['key2'] = df['key2'].astype('category')
# ``이제 'key' 열의 데이터 타입이 'category'로 변환되었습니다.
# df.head()
# df2 = df[["Machinery", 'Assembly']]
# df2

In [5]:
# sns.countplot(x='key2', data=df)

In [6]:
# df['key2'].mode()
# grouped_df = df.groupby('key2')
# grouped_df.head()
# grouped_df[['Subject','Machinery', 'Assembly', '청구품목']].head()
# series1 = grouped_df['청구서번호'].count()
# series1.iloc[:,]

In [7]:

# plt.bar(grouped_df['청구서번호'].count().index,grouped_df['청구서번호'].count())
# plt.xlabel("key2")
# plt.ylabel('count')
# plt.xticks(rotation=45)
# plt.ylim(500,4200)
# # 특정 품목만 많이 청구됨

In [13]:
df['key2'].shape

(20517,)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

le_x = preprocessing.LabelEncoder()
# df = df.apply(le.fit_transform)

X = df[['Subject','Machinery', 'Assembly' , "Part No.1","청구품목"]]
X = X.apply(le_x.fit_transform)

y = df["key2"]
le_y = preprocessing.LabelEncoder()
y = le_y.fit_transform(y)
# y_encoded = le_y.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40, shuffle=True, test_size=0.2)

In [23]:
X.iloc[15:21,:]

,Subject,Machinery,Assembly,Part No.1,청구품목
15,418,26,1390,4821,5510
16,418,26,1390,5134,4916
17,418,26,1390,5760,5510
18,1272,148,1248,1703,1899
19,994,148,1248,1703,1887
20,1272,148,1248,1833,2544


In [24]:
y

array([18, 18, 18, ...,  5, 20, 20])

# CategoricalNB

In [11]:
import numpy as np
from sklearn.naive_bayes import CategoricalNB
 
rng = np.random.RandomState(100)
clf = CategoricalNB(alpha=0.1).fit(X_train, y_train) 
## alpha : Laplace Smoothing 파라미터

In [12]:
y_pred = clf.predict(X_train)
y_pred

array([20, 33, 45, ..., 40, 50, 57])

In [13]:
print(clf.get_params()) ## CategoricalNB 클래스 인자 설정 정보
print('정확도 : ', clf.score(X_train,y_train)) ## 성능 평가 점수(Accuracy
print('정확도 : ', clf.score(X_test,y_test)) ## 성능 평가 점수(Accuracy

{'alpha': 0.1, 'class_prior': None, 'fit_prior': True, 'force_alpha': 'warn', 'min_categories': None}
정확도 :  0.968561506123195
정확도 :  0.7775341130604289


In [14]:
clf.score(X_train, y_train)

0.968561506123195

In [15]:
from sklearn.metrics import f1_score
# 모델 평가
from sklearn.metrics import mean_squared_error, r2_score
y_pred = clf.predict(X)
mse = mean_squared_error(y, y_pred)


# 모델 예측
y_pred = clf.predict(X_test)

# mse 값 계산
print("MSE: ", mse)

# F1-score 계산
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score:", f1)


MSE:  36.343276307452356
F1-score: 0.779383691454742


# OneVsRestClassifier

# LogisticRegression

## Tree

In [3]:
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

df = pd.read_csv('../data/raw_postpro.csv')

df = df.apply(preprocessing.LabelEncoder().fit_transform)
X = df[['Machinery', 'Assembly' , "Part No.1","청구품목"]]
y = df["key2"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40, shuffle=True, test_size=0.2)

In [4]:
tree = DecisionTreeClassifier(max_depth=24, random_state= 0) # 트리 생성
tree.fit(X_train, y_train)
print("훈련 세트 정확도 : {:.3f}".format(tree.score(X_train,y_train)))
print("테스트 세트 정확도 : {:.3f}".format(tree.score(X_test,y_test))) 

훈련 세트 정확도 : 0.974
테스트 세트 정확도 : 0.755


In [5]:
from sklearn.metrics import f1_score

# 모델 예측
y_pred = tree.predict(X_test)

# F1-score 계산
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score:", f1)


F1-score: 0.7540515071387015


## Xgboost : softmax

In [25]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# xgboost 모델 생성
model = xgb.XGBClassifier(objective='multi:softmax', num_class=61)

# 모델 학습
model.fit(X_train, y_train)

# 예측 결과 출력
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.831140350877193


In [26]:
from sklearn.metrics import f1_score

# 모델 예측
y_pred = model.predict(X_test)

# F1-score 계산
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score:", f1)

F1-score: 0.8294482818795842


In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer


from xgboost import XGBClassifier
from xgboost import plot_importance

import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
# 텍스트 데이터
df = pd.read_csv('../data/raw_postpro.csv', encoding = 'cp949')
X = list(df['Machinery']  + ' ' + df['Assembly'] + ' ' + df['청구품목'] + ' ' + df['Part No.1'])
y = df["key2"]

In [50]:
# CountVectorizer를 이용하여 feature 벡터 생성
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

In [51]:
# y 값을 숫자로 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [52]:
type(y)

unique_sorted_array = np.sort(np.unique(y))

# 결과 출력
print(unique_sorted_array)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60]


In [30]:
# 학습 데이터와 테스트 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# xgboost 모델 생성
model = XGBClassifier(objective='multi:softmax', num_class=61)

# 모델 학습
model.fit(X_train, y_train)

# 예측 결과 출력
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8581871345029239


In [31]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score
def print_metrics(y_test, X_test, model) :
  # 테스트 데이터셋으로 예측
  y_pred = model.predict(X_test)

  # 각종 평가지표 계산
  recall = recall_score(y_test, y_pred, average='macro')
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='macro')
  f1 = f1_score(y_test, y_pred, average='macro')

  print("Recall: {:.4f}".format(recall))
  print("Accuracy: {:.4f}".format(accuracy))
  print("Precision: {:.4f}".format(precision))
  print("F1-score: {:.4f}".format(f1))


In [32]:
 print_metrics(y_test, X_test, model)

Recall: 0.8235
Accuracy: 0.8582
Precision: 0.8675
F1-score: 0.8404


In [9]:
# import tensorflow as tf
# print(tf.__version__)

2.12.0


In [55]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression 모델 생성 및 학습
# lr = LogisticRegression(random_state=42, C=7, solver="liblinear", penalty='l1', multi_class="ovr", max_iter=1000)
lr = LogisticRegression(random_state=42, C=14, solver="liblinear", penalty='l1', multi_class= "auto", max_iter=1000)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)
print_metrics(y_test, X_test, lr)

Recall: 0.8636
Accuracy: 0.9040
Precision: 0.9209
F1-score: 0.8832


# svc model

In [56]:
from sklearn.svm import SVC

svc = SVC(kernel='linear', decision_function_shape='ovr')
svc.fit(X_train, y_train)

SVC(kernel='linear')

In [57]:
svc_pred = svc.predict(X_test)
svc_acc = accuracy_score(y_test, svc_pred)
print_metrics(y_test, X_test, svc)

Recall: 0.8271
Accuracy: 0.8667
Precision: 0.8836
F1-score: 0.8449


# RandomForestClassifier

In [58]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [59]:
rf_pred = rf.predict(X_test)
rf_acc = accuracy_score(y_test, rf_pred)
print_metrics(y_test, X_test, rf)

Recall: 0.7730
Accuracy: 0.8065
Precision: 0.8549
F1-score: 0.8022


# DecisionTreeClassifier

In [60]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [61]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [62]:
y_pred = dt.predict(X_test)

# 정확도, 정밀도, 재현율, F1 점수를 계산합니다.
print(classification_report(y_test, y_pred))

# 혼동 행렬을 출력합니다.
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.40      0.44        40
           1       1.00      0.57      0.73         7
           2       0.59      0.59      0.59        17
           3       0.67      0.40      0.50        10
           4       0.86      0.87      0.87       215
           5       0.89      0.80      0.84        10
           6       1.00      1.00      1.00        13
           7       0.92      0.96      0.94        23
           8       0.69      0.66      0.67       175
           9       0.50      0.20      0.29         5
          10       0.96      0.90      0.93        50
          11       1.00      1.00      1.00         3
          12       0.74      0.59      0.66        39
          13       0.69      0.65      0.67        17
          14       1.00      0.43      0.60         7
          15       0.75      0.60      0.67         5
          16       0.83      0.94      0.88        16
          17       0.74    

In [10]:
# import tensorflow as tf

# print("TensorFlow version:", tf.__version__)
# print("Keras version:", tf.keras.__version__)

TensorFlow version: 2.12.0
Keras version: 2.12.0


In [11]:
# !pip install gensim

  Using cached gensim-4.3.1-cp310-cp310-win_amd64.whl (24.0 MB)
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)


In [102]:
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# 4개 컬럼을 하나의 문자열로 합치기
data = df['Machinery'] + ' ' + df['Assembly'] + ' ' + df['청구품목'] + ' ' + df['Part No.1']
y = df["key2"]

# 문장 토큰화
sentences = [text_to_word_sequence(sentence) for sentence in data]

# Word2Vec 모델 학습
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4) # 벡터사이즈 100

# 단어 임베딩 활용
embedded_data = []
for sentence in sentences:
    embedded_sentence = [model.wv[word] for word in sentence]
    embedded_data.append(embedded_sentence)

In [103]:
# embedded_data
# 문장들의 길이 확인
sentence_lengths = [len(sentence) for sentence in embedded_data]

# 가장 긴 문장의 길이 확인
max_sequence_length = max(sentence_lengths)

print("가장 긴 문장의 길이:", max_sequence_length)

가장 긴 문장의 길이: 28


In [104]:
# y 값을 숫자로 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [105]:
X,y

(<20517x11320 sparse matrix of type '<class 'numpy.int64'>'
 	with 221653 stored elements in Compressed Sparse Row format>,
 array([18, 18, 18, ...,  5, 20, 20]))

In [106]:
# train, test 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embedded_data, y, test_size=0.2, random_state=42)

In [107]:
# type(X_train)
# print([len(x) for x in X_train])
max_length = max([len(x) for x in X_train])

# 모든 원소들이 동일한 길이로 처리되도록 길이를 조절하거나 자른다.
# X_train_padded = []
# for x in X_train:
#     if len(x) < max_length:
#         x += [0] * (max_length - len(x))  # 0으로 패딩
#     elif len(x) > max_length:
#         x = x[:max_length]  # 일정 길이를 초과하는 부분 자르기
#     X_train_padded.append(x)

X_train_padded = []
for x in X_train:
    padding_shape = (max_length - len(x), 100)  # 패딩할 행렬 모양 확인
    padding_matrix = np.zeros(padding_shape)  # 해당 모양의 0 행렬 생성
    if len(x) < max_length:
        x = np.concatenate((x, padding_matrix), axis=0)  # 0으로 패딩
    elif len(x) > max_length:
        x = x[:max_length]  # 일정 길이를 초과하는 부분을 자르기
    X_train_padded.append(x)

X_train_array = np.array(X_train_padded)

In [108]:
# X_train_array
X_test

X_test_padded = []
for x in X_test:
    padding_shape = (max_length - len(x), 100)  # 패딩할 행렬 모양 확인
    padding_matrix = np.zeros(padding_shape)  # 해당 모양의 0 행렬 생성
    if len(x) < max_length:
        x = np.concatenate((x, padding_matrix), axis=0)  # 0으로 패딩
    elif len(x) > max_length:
        x = x[:max_length]  # 일정 길이를 초과하는 부분을 자르기
    X_test_padded.append(x)
X_test_array = np.array(X_test_padded)

In [109]:
# print('X_train_array shape:', X_train_array.shape)
# print('X_train_array dtype:', X_train_array.dtype)
# print('y_train_array shape:', y_train_array.shape)
# print('y_train_array dtype:', y_train_array.dtype)
print('X_train_array shape:', X_train_array.shape)
print('X_train_array dtype:', X_train_array.dtype)
print('X_test_array shape:', X_test_array.shape)
print('X_test_array dtype:', X_test_array.dtype)
print('y_train shape:', y_train.shape)
print('y_train dtype:', y_train.dtype)

X_train_array shape: (16413, 28, 100)
X_train_array dtype: float64
X_test_array shape: (4104, 28, 100)
X_test_array dtype: float64
y_train shape: (16413,)
y_train dtype: int32


In [116]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pandas as pd
import tensorflow as tf
from scipy.sparse import coo_matrix
import numpy as np
from scipy.sparse import coo_matrix
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# X_train_array = np.asarray(X_train, dtype=np.float32)

# y_train_array = np.asarray(y_train_array, dtype=np.float32)
y_train_array_encoded = to_categorical(y_train)
y_test_array_encoded = to_categorical(y_test)

# Reshape X_train_array to be 2D
X_train_array = X_train_array.reshape(X_train_array.shape[0], -1)

# X_train과 y_train 데이터를 희소 행렬로 변환
# X_train_sparse = coo_matrix(X_train_array)
# y_train_sparse = coo_matrix(y_train_array)
# 모델 정의
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_array.shape[1],)))
model.add(Dense(64, activation='relu'))
# model.add(Dense(1, activation='linear'))

# 범주수에 해당하는 뉴런을 추가하고, 활성화 함수를 softmax로 변경합니다.
model.add(Dense(61, activation='softmax'))

# 모델 컴파일 및 학습
# model.compile(loss='mean_squared_error', optimizer='adam')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_array, y_train_array, epochs=10, batch_size=32)

Epoch 1/10


ValueError: in user code:

    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "D:\Shi

In [90]:
y_test

array([54, 33, 24, ..., 51, 25, 40])

In [91]:
# X_test_array
X_test_array = X_test_array.reshape(X_test_array.shape[0], -1)

In [92]:
y_pred = model.predict(X_test_array)

129/129 [==============================] - 0s 3ms/step


In [75]:
y_pred = y_pred.reshape(-1)

In [93]:
y_pred

array([[55.083607],
       [31.186554],
       [21.126211],
       ...,
       [23.491358],
       [30.733864],
       [41.91265 ]], dtype=float32)

In [97]:
rounded_y_pred = np.round(y_pred)
f1 = f1_score(y_test, rounded_y_pred, average='macro')
print("F1 Score:", f1)
# f1 = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
# f1, accuracy

F1 Score: 0.032947391607001296


In [100]:
accuracy = accuracy_score(y_test, rounded_y_pred)

In [101]:
accuracy

0.06115984405458089

In [42]:
# from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score
# Logistic Regression 모델 생성 및 학습
# lr = LogisticRegression(random_state=42, C=14, solver="liblinear", penalty='l1', multi_class="ovr", max_iter=1000)
# lr.fit(X_train, y_train)
# y_pred = model.predict(X_test)

#   # 각종 평가지표 계산
# recall = recall_score(y_test, y_pred, average='macro')
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

KeyboardInterrupt: 

In [ ]:
print("Recall: {:.4f}".format(recall))
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("F1-score: {:.4f}".format(f1))


In [20]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score
def print_metrics(y_test, X_test, model) :
  # 테스트 데이터셋으로 예측
  y_pred = model.predict(X_test)

  # 각종 평가지표 계산
  recall = recall_score(y_test, y_pred, average='macro')
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='macro')
  f1 = f1_score(y_test, y_pred, average='macro')

  print("Recall: {:.4f}".format(recall))
  print("Accuracy: {:.4f}".format(accuracy))
  print("Precision: {:.4f}".format(precision))
  print("F1-score: {:.4f}".format(f1))
